In [1]:
!pip install --upgrade pip
!pip install numpy~=1.0  # 1.0 instead of 2.0 for pymatching compatibility later
!pip install scipy
!pip install stim~=1.14
!pip install pymatching~=2.0
!pip install qiskit

In [2]:
import stim
print(stim.__version__)

1.14.0


In [3]:
import stim
import pymatching
import numpy as np
def count_logical_errors(circuit: stim.Circuit, num_shots: int) -> int:
    # Sample the circuit.
    sampler = circuit.compile_detector_sampler()
    detection_events, observable_flips = sampler.sample(num_shots, separate_observables=True)

    # Configure a decoder using the circuit.
    detector_error_model = circuit.detector_error_model(decompose_errors=True)
    matcher = pymatching.Matching.from_detector_error_model(detector_error_model)

    # Run the decoder.
    predictions = matcher.decode_batch(detection_events)

    # Count the mistakes.
    num_errors = 0
    for shot in range(num_shots):
        actual_for_shot = observable_flips[shot]
        predicted_for_shot = predictions[shot]
        if not np.array_equal(actual_for_shot, predicted_for_shot):
            num_errors += 1
    return num_errors

In [4]:
import stim
def get_logical_err_rate(distance: int, rounds: int, phy_err_p: int, num_shots: int) -> float:
    # Generate Surface Code circuit with the specified parameters.
    circuit = stim.Circuit.generated(
            "surface_code:rotated_memory_z",
            rounds=rounds,
            distance=distance,
            after_clifford_depolarization=phy_err_p,
            after_reset_flip_probability=phy_err_p,
            before_measure_flip_probability=phy_err_p,
            before_round_data_depolarization=phy_err_p,
        )
    return count_logical_errors(circuit, num_shots) / num_shots

In [5]:
print(get_logical_err_rate(distance=7, rounds=21, phy_err_p=0.001, num_shots=100000))

8e-05


In [6]:
def get_distance_from_logical_err_rate(logical_err_rate: float, phy_err_p: int) -> int:
    # Binary search for the distance that gives the desired logical error rate.
    num_shots = int(10 / logical_err_rate)
    high = 2
    while get_logical_err_rate(high, high * 3, phy_err_p, num_shots) > logical_err_rate:
        high *= 2
    low = max(high // 2, 2)
    while low < high:
        mid = (low + high) // 2
        err_rate = get_logical_err_rate(mid, mid * 3, phy_err_p, num_shots)
        if err_rate < logical_err_rate:
            high = mid
        else:
            low = mid + 1
    return high

In [7]:
print(get_distance_from_logical_err_rate(logical_err_rate=0.0001, phy_err_p=0.001))

7


In [8]:
class PhysicalQubit:
    def __init__(self, gate_error_rate, measurement_error_rate, 
                 single_qubit_gate_time=50e-9, two_qubit_gate_time=100e-9, measurement_time=2600e-9):
        self.single_qubit_gate_time = single_qubit_gate_time
        self.two_qubit_gate_time = two_qubit_gate_time      
        self.gate_error_rate = gate_error_rate              
        self.measurement_time = measurement_time
        self.measurement_error_rate = measurement_error_rate

In [9]:
class SurfaceCode:
    def __init__(self, code_distance, physical_qubit_params):
        self.code_distance = code_distance
        self.physical_qubit = physical_qubit_params
        self.logical_cycle_time = 3 * self.code_distance * (4 * self.physical_qubit.two_qubit_gate_time + 2 * self.physical_qubit.measurement_time)

    def logical_error_rate(self):
        p_physical = self.physical_qubit.gate_error_rate
        return p_physical ** (self.code_distance / 2)

    def physical_qubits_per_logical(self):
        return 2 * self.code_distance ** 2 - 1

In [10]:
class TFactory:
    def __init__(self, surface_code, distillation_level=1):
        self.surface_code = surface_code
        self.distillation_level = distillation_level
        self.t_round = 10 * self.surface_code.logical_cycle_time
        self.t_distillation = self.distillation_level * self.t_round
        self.t_gate_per_second = 1 / self.t_distillation

    def required_physical_qubits(self, max_runtime, t_count):
        required_t_per_second = t_count / max_runtime
        num_t_factories = int(np.ceil(required_t_per_second / self.t_gate_per_second))
        num_logical_qubits = num_t_factories * 15
        return num_logical_qubits * self.surface_code.physical_qubits_per_logical()

    def distillation_time(self):
        return self.t_distillation

In [11]:
class Algorithm:
    def __init__(self, t_gates, one_q_gates, two_q_gates, qubits, total_cycles):
        self.t_gates = t_gates         
        self.one_q_gates = one_q_gates
        self.two_q_gates = two_q_gates
        self.qubits = qubits
        self.total_cycles = total_cycles

In [12]:
class ResourceEstimator:
    def __init__(self, algorithm, surface_code, t_factory):
        self.algorithm = algorithm
        self.surface_code = surface_code
        self.t_factory = t_factory

    def estimate(self):
        logical_qubits = self.algorithm.qubits
        total_time = self.algorithm.total_cycles * self.surface_code.logical_cycle_time
        total_physical = self.t_factory.required_physical_qubits(total_time, self.algorithm.t_gates) + logical_qubits * self.surface_code.physical_qubits_per_logical()
        return {
            "logical_qubits": logical_qubits,
            "physical_qubits": total_physical,
            "total_time_seconds": total_time
        }

In [15]:
import qiskit
from qiskit import QuantumCircuit

def estimate_circuit_cost(circ: QuantumCircuit):
    num_qubits = circ.num_qubits
    num_gates = circ.size()
    logical_cycles = circ.depth()
    print(f"num qubits: {num_qubits}, num gates: {num_gates}, logical cycles: {logical_cycles}")
    logical_error_rate = 0.5 / logical_cycles
    distance = get_distance_from_logical_err_rate(logical_error_rate, 3.36e-3)
    print(f"code distance: {distance}")
    physical_qubit = PhysicalQubit(gate_error_rate=3.36e-3, measurement_error_rate=7.813e-3)
    surface_code = SurfaceCode(code_distance=distance, physical_qubit_params=physical_qubit)
    t_factory = TFactory(surface_code=surface_code, distillation_level=1)
    t_gates = circ.count_ops().get('t', 0)
    one_q_gates = sum([circ.count_ops().get(op, 0) for op in ['x', 'y', 'z', 'h', 's', 't']])
    two_q_gates = sum([circ.count_ops().get(op, 0) for op in ['cp', 'cx', 'swap']])
    print(f"t gates: {t_gates}, one q gates: {one_q_gates}, two q gates: {two_q_gates}, qubits: {num_qubits}, logical cycles: {logical_cycles}")
    algorithm = Algorithm(t_gates=t_gates, one_q_gates=one_q_gates, two_q_gates=two_q_gates, total_cycles=circ.depth(), qubits=circ.num_qubits)
    estimator = ResourceEstimator(algorithm, surface_code, t_factory)
    result = estimator.estimate()    
    print(f"logical qubit num: {result['logical_qubits']}")
    print(f"physical qubit num: {result['physical_qubits']}")
    print(f"runtime: {result['total_time_seconds'] * 1e9} nanoseconds")

In [16]:
import qiskit
from qiskit import QuantumCircuit
from qiskit import qpy
global syned_qc
with open('syned_qc.qpy', 'rb') as qc_file:
    syned_qc = qpy.load(qc_file)[0]
estimate_circuit_cost(syned_qc)

num qubits: 10, num gates: 3975, logical cycles: 1385
code distance: 16
t gates: 1621, one q gates: 3885, two q gates: 90, qubits: 10, logical cycles: 1385
logical qubit num: 10
physical qubit num: 97090
runtime: 372288000.00000006 nanoseconds
